# MySQLこと始めメモ

### ターミナル
```sh
mysql.server start```
    サーバー起動　これで各種機構からログインが可能になる
```sh
mysql.server stop``` 
    サーバー停止　終わるときはこうしよう
```sh
mysql -u root``` 
    rootユーザーとしてターミナル上でmysqlにログイン
    
## mysql>
```mysql
show databases;```
    すべてのデータベースを閲覧(git branchのノリ)
```mysql
create database A;```
    Aのデータベースを作成(git branch Aのノリ)
```mysql
use A;```
    Aのデータベースに移動(git checkout Aのノリ)
    
(mysqlでは;が行末なので，忘れずにつけること)

### PyMySQL
```Python 
connection = pymysql.connect(OPTION)```
    はじめに書こう　OPTIONの情報を用いてデータベースに接続  

>### OPTION
- #### host='localhost’,  
接続先を指定するんだと思う　どこにあるmysqlですか？的なやつかな  
- #### user='root’,  
mysqlにログインする時のユーザー名  
- #### password=‘’,  
mysqlにログインする時のパスワード  
- #### db='test_ocw’,  
使うデータベース  
- #### charset='utf8’,  
普通に文字コードだと思う  
- #### cursorclass=pymysql.cursors.DictCursor  
DictCursorで「Selectの結果をdictionary形式で受け取る」らしい  

```Python 
with connection.cursor() as cursor:```
    cursorの立ち上げ(withでやった方がclose処理を書かずに済み，楽)
```Python 
cursor.execute("SQL文")```
    SQL文の実行
```Python 
connection.commit()```
    変更の保存，スクレイピングして格納した後とかに使おう
```Python 
cursor.fetchall()```
    (直前にcursor.executeでSELECT文を送ってる必要あり)SELECTで入手した結果をすべて取得　他には呼ぶ度1つずつ取ってくるfetchoneなどが存在

```Python
cursor.execute("show columns from lecture")
for rows in cursor.fetchall(): print(rows["Field"])
```
    全キー取得　主にめんどい人用

(DictCursorにするとSELECT→fetch時にkeyが属性名の辞書が帰ってくるので，Django的にはこれが楽な気がする)

In [1]:
#データベースに接続
import pymysql
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='test_ocw',
                             charset='utf8',
                             # Selectの結果をdictionary形式で受け取る
                             cursorclass=pymysql.cursors.DictCursor)

In [12]:
with connection.cursor() as cursor:
    sql = """CREATE TABLE `L` (
    `CODE` CHAR(10) NOT NULL,
    `lecture name` TEXT NOT NULL,
    `professor` TEXT NOT NULL,
    PRIMARY KEY (`CODE`));"""
    cursor.execute(sql)

In [15]:
#SELECT文を実行してデータを見る
with connection.cursor() as cursor:
    sql = "SELECT * FROM L"
    cursor.execute(sql)
 
    dbdata = cursor.fetchall()
    for rows in dbdata:
        print(rows)

{'CODE': '111111', 'lecture name': 'おもろい授業', 'professor': 'おもろい生活'}


In [16]:
#INSERT文を実行してデータを追加し，再びSELECT文の実行して確認する
with connection.cursor() as cursor:
    sql = """INSERT INTO `L`
    (`CODE`, `lecture name`, `professor`) VALUES 
    ('999999','すばらしい授業','すばらしい生活')"""
    cursor.execute(sql)
    
    sql = "SELECT * FROM L"
    cursor.execute(sql)
 
    dbdata = cursor.fetchall()
    for rows in dbdata:
        print(rows)

{'CODE': '111111', 'lecture name': 'おもろい授業', 'professor': 'おもろい生活'}
{'CODE': '999999', 'lecture name': 'すばらしい授業', 'professor': 'すばらしい生活'}


In [17]:
#変更の保存
connection.commit()

In [18]:
#お　ま　け
#「SQL文を動的にかきたい！」
#対話形式でテーブルにデータを入れるスクリプト

TABLE_NAME = "L"

#VALUE_TAPLE,KEY_TAPLEの作成
with connection.cursor() as cursor:
    keys,values = [],[]
    #show columns実行
    sql = "show columns from {}".format(TABLE_NAME)
    cursor.execute(sql)
    
    for rows in cursor.fetchall():
        k = rows["Field"]
        keys.append(k)
        #####入力#####
        print("`{}` value?".format(k),end=">")
        values.append(input())
        #############
        
    VALUE_TAPLE = "({})".format(",".join(["\"{}\"".format(v) for v in values]))
    KEY_TAPLE = "({})".format(",".join(["`{}`".format(k) for k in keys]))

#INSERTしてINSERT後のテーブルを閲覧
with connection.cursor() as cursor:
    #INSERT実行
    sql = "INSERT INTO {} {} VALUES {}"\
        .format(TABLE_NAME,KEY_TAPLE,VALUE_TAPLE)
    cursor.execute(sql)
    
    #SELECT実行
    sql = "SELECT * FROM {}".format(TABLE_NAME)
    cursor.execute(sql)
 
    dbdata = cursor.fetchall()
    for rows in dbdata:
        print(rows)

`CODE` value?>893
`lecture name` value?>過激な授業
`professor` value?>過激な生活
{'CODE': '111111', 'lecture name': 'おもろい授業', 'professor': 'おもろい生活'}
{'CODE': '893', 'lecture name': '過激な授業', 'professor': '過激な生活'}
{'CODE': '999999', 'lecture name': 'すばらしい授業', 'professor': 'すばらしい生活'}
